## 자연어 처리기로 데이터 셋 만들기

In [1]:
# 데이터 핸들링 프레임
import numpy as np 
import pandas as pd
from pandas import DataFrame as df
import csv
import os

# 자연어 처리기
from konlpy.tag import Okt
from collections import Counter

# word2Vec
import gensim 
from gensim.models import word2vec

In [2]:
dataset = pd.read_csv("total_dataset_trim_newTag_scale.csv",encoding='utf-8')
# dataset.tail()

In [113]:
## description 다듬기 : 데이터 정규화, 맥락 위주로 자르고 명사만 가져오기, 글자길이는 1 ~ 7  
def getDescList(n):
    okt=Okt()
    descList=[]
    for i in range(n):
        tempList=[]
        descript=(dataset.description[i]) # 한글자 이상만 뽑음
        descript=descript.replace("\n","")
        descript=descript.replace("\r","")
        descript=descript.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣AZ09]","")
        tempList=[j[0] for j in okt.pos(descript) if ((len(j[0])>1)& (len(j[0])<7)& (j[1]=="Noun"))]
        tempList=list(set(tempList))
        if len(tempList)>0:
            descList.append(tempList)
        else:
            descList.append(tempList)
    return descList

In [114]:
## Tag 다듬기 : 복합명사로 이루어진 태그는 잘라서 길이 2~7 사이의 단어만 추출
def getTagList(n):
    parseTagList=[]
    okt=Okt()
    for i in range(n):
        tempTagList=[]
        tags=str(dataset.tag[i]).split('#') # 태그별로 자르기
        for tag in tags:
            if len(tag)>1:
                okt.pos(tag)  # 1개 태그 내 복합 명사 자르기
                for j in okt.nouns(tag):
                    if(len(j)>=2|len(j)<7):
                        tempTagList.append(j)
        tempTagList=list(set(tempTagList))
        parseTagList.append(tempTagList)
    return parseTagList

In [105]:
## Case 1) 코어 태그 추출 (태그, description 합집합)
def getCoreTags1(n):
    coreTagList1=[]
    okt=Okt()    
    coreTag=[]
    descript=getDescList(n) # 한글자 이상만 뽑음
    tags=getTagList(n)  # tag를 한번 더 파싱
    for i in range(len(descript)): 
        if (len(tags[i])>0): 
            coreTag.append(tags[i]+descript[i])
        else:
            coreTag.append(descript[i])
    coreTagList1.append(coreTag)
    return coreTagList1

%time CoreTagDate1=getCoreTags1(len(dataset))
CoreTagDate1=CoreTagDate1[0]

Wall time: 40.2 s


In [106]:
## Case 2) 코어 태그 추출 (부분적으로 기존 태그 + description + coreTag )
def getCoreTags2(n):
    coreTagList=[]
    okt=Okt()    
    descript=getDescList(n) # 한글자 이상만 뽑음
    tags=getTagList(n)  # tag를 한번 더 파싱
    for k in range(len(descript)):
        coreTags=[]
        for t in range(len(tags[k])):
            if (len(tags[k][t])>0)&(tags[k][t] in descript[k]): # 태그가 description 단어에도 있는지 확인
                coreTags.append(tags[k][t])
        if (len(tags[k])<=3): # 태그 수가 없으면 디스크립션으로
            coreTagList.append(list(set(descript[k])))
        elif((len(coreTags)<=3)&(len(tags[k])>3)):# desc와 일치하는 태그(coreTag)가 적은데 기존 태그(tags[k])많으면 기존 태그 사용
            coreTagList.append(tags[k])
        elif(len(coreTags)>3): # 디스크립션과 일치하는 태그가 많으면 일치태그 사용
            coreTagList.append(coreTags)
        else:
            coreTagList.append(list(set(descript[k])))
            print("descript[k]",descript[k])
    return coreTagList
%time CoreTagDate2=getCoreTags2(1852)

Wall time: 1min 25s


In [107]:
# Tag2 데이터 보정 : Tag1의 데이터 갖다쓰기
for i in range(len(CoreTagDate2)):
    if len(CoreTagDate2[i])<1:
        print(CoreTagDate2[i],i)
        CoreTagDate2[i]=list(set(CoreTagDate1[i]))

[] 92
[] 1486
[] 1806


In [108]:
for i in range(len(CoreTagDate2)):
    CoreTagDate2[i]=list(set(CoreTagDate2[i]))
    CoreTagDate1[i]=list(set(CoreTagDate1[i]))


## 모델 가동, 그 결과를 새로운 태그로 (extend_tag) 컬럼으로 저장

In [109]:
# Word2Vec 모델
num_features =350    # Word vector dimensionality                      
min_word_count =2   # Minimum word count                        
num_workers = 2     # Number of threads to run in parallel
context = 4          # Context window size                                                                                    
downsampling = 1e-3  # Downsample setting for frequent words
dataset1=dataset

model1 = gensim.models.Word2Vec(CoreTagDate1, workers=num_workers, 
                          size=num_features, min_count = min_word_count,
                          window = context, sample = downsampling)
model_name = "TagSimilarity1"
model1.save(model_name)

In [93]:
# model.wv.vocab
test_keyword=["가정","사진","촬영","원목","아파트","루프","자연","채광","스튜디오","광고","거실","주택","파티","빌라","카페",
              "빈티","플라워","홍대","공간","화이트","느낌","분위기","원룸",
              "사무실","서울","마당","정원","모던","인테리어","오피스"]

testkeywordDict1={}
for keyword in test_keyword:
    testkeywordDict1.update({keyword:model1.wv.most_similar(keyword,topn=5)})
testkeywordDict1

{'가정': [('아파트', 0.9999679923057556),
  ('화이트', 0.9999654293060303),
  ('복층', 0.9999648332595825),
  ('채광', 0.9999646544456482),
  ('미니', 0.9999638795852661)],
 '사진': [('연출', 0.9999827146530151),
  ('문의', 0.9999758005142212),
  ('저희', 0.9999749064445496),
  ('드라마', 0.9999745488166809),
  ('예약', 0.9999743700027466)],
 '촬영': [('영상', 0.9999771118164062),
  ('사진', 0.9999743103981018),
  ('드라마', 0.9999740123748779),
  ('제품', 0.9999732375144958),
  ('문의', 0.9999725818634033)],
 '원목': [('느낌', 0.9999755024909973),
  ('거실', 0.9999753832817078),
  ('모두', 0.9999713897705078),
  ('소품', 0.9999698400497437),
  ('우드', 0.999968409538269)],
 '아파트': [('화이트', 0.9999776482582092),
  ('조명', 0.999975860118866),
  ('시간', 0.9999750852584839),
  ('가구', 0.9999749064445496),
  ('건물', 0.9999733567237854)],
 '루프': [('위치', 0.9999781847000122),
  ('영화', 0.9999741315841675),
  ('바로', 0.9999725222587585),
  ('감성', 0.9999699592590332),
  ('장소', 0.9999698400497437)],
 '자연': [('장소', 0.9999760389328003),
  ('광고', 0.9999748

In [133]:
## Tag1 데이터 셋을 기반으로 모델 작성, Tag2 데이터 셋의 유사단어 리스트 추출
def extendTag():
    vocab=model1.wv.vocab
    extendTagList=[]
    extendTagSimList=[]
    for i in range(len(CoreTagDate2)):
        tags=[]
        tagsSim=[]
        for j in range(len(CoreTagDate2[i])):
            if((len(CoreTagDate2[i])>25)):
                tags.append(CoreTagDate2[i][j])
                tagsSim.append(0)
            if((CoreTagDate2[i][j] in vocab.keys())&(len(CoreTagDate2[i])>12)):
                for k in range(1):
                    tags.append(model1.wv.most_similar(CoreTagDate2[i][j],topn=1)[k][0])
                    tagsSim.append(model1.wv.most_similar(CoreTagDate2[i][j],topn=1)[k][1])
            elif((CoreTagDate2[i][j] in vocab.keys())&(len(CoreTagDate2[i])>5)):
                for k in range(3):
                    tags.append(model1.wv.most_similar(CoreTagDate2[i][j],topn=3)[k][0])
                    tagsSim.append(model1.wv.most_similar(CoreTagDate2[i][j],topn=3)[k][1])
            elif((CoreTagDate2[i][j] in vocab.keys())&(len(CoreTagDate2[i])<=4)):
                for k in range(4):
                    tags.append(model1.wv.most_similar(CoreTagDate2[i][j],topn=4)[k][0])                    
                    tagsSim.append(model1.wv.most_similar(CoreTagDate2[i][j],topn=4)[k][1])
        extendTagList.append(list(set(tags)))
        extendTagSimList.append(list(set(tagsSim)))
    return extendTagList,extendTagSimList
extend_tag=extendTag()

In [137]:
dataset["extend_tag"]=extend_tag[0]
dataset["extend_tag_sim"]=extend_tag[1]

In [138]:
# 유사단어 컬럼 추가한 파일 저장
dataset.to_csv('C:/바탕 화면/playdata/ice/eclipse/Devils_Project/dataWork/controller/total_dataset_trim_newExtendTag.csv',
                 sep=',',na_rep='NaN',index =True,encoding='utf-8-sig')

## 키워드 카운트
    여기서부터는 Controller의 영역

In [ ]:
dataset = pd.read_csv("total_dataset_trim_newExtendTag.csv",encoding='utf-8')

In [172]:
#임의로 1개 선택
target=dataset.iloc[:1]
rest=dataset.iloc[1:]
# target=dataset.iloc[k:k+1]
# rest=dataset.iloc[k+1:]


# 문자열로 저장된 태그 리스트로 변환
def strToList(value):
    if type(value)==str:
        trimList=value.replace(']','')
        trimList=trimList.replace('[','')
        trimList=trimList.replace("'",'')
        trimList=trimList.split(",")
        trimList=list(set(trimList))  # 리스트 안에서 중복되는 태그 삭제
    else:
        trimList=list(set(value))
    return trimList

## 유사 태그 개수 세기
def tagCount(targetTag):
    countKeyword=[]
    for i in range(len(rest["extend_tag"])):
        cnt=0
        for j in range(len(strToList(targetTag))):
            for k in range(len(strToList(reset["extend_tag"]))):
                if targetTag[j] == strToList(rest.iloc[i]["extend_tag"]):
                    weight=rest.iloc[j,["extend_tag_sim"]]
                    cnt+=1*weight
        countKeyword.append(cnt)
    return countKeyword

tagCount=tagCount(target["extend_tag"].values[0])
# rest["tagCount"]=tagCount
# target["tagCount"]=0  # target은 아예 안 뜨도록 0으로 처리

# 다시 2개 합치기
# dataset=pd.concat([target,rest])
# dataset1=dataset

IndexError: .iloc requires numeric indexers, got ['extend_tag_sim']

In [ ]:
target["extend_tag"].values[0]